# MLP Model (Regression)

In [ ]:
import non_time_series_utils as utils
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Import data instead of generation
df = utils.generate_df()

In [ ]:
train_df, test_df = utils.split_df_to_train_test(df, split_rate=0.6)
test_df, val_df = utils.split_df_to_train_test(test_df, split_rate=0.5)

In [ ]:
train_X, train_y = utils.split_df_to_X_y(train_df)
val_X, val_y = utils.split_df_to_X_y(val_df)

In [ ]:
scaler = StandardScaler().fit(train_X)

In [ ]:
def build_model(input_shape, n_hidden_layers, n_neurons):
    model = Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.001), bias_regularizer=regularizers.l2(0.001)))

    for i in range(0, n_hidden_layers):
        model.add(Dropout(0.5))
        model.add(Dense(n_neurons, activation='relu', kernel_regularizer=regularizers.l2(0.001), bias_regularizer=regularizers.l2(0.001)))

    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
n_hidden_layers_list = [1, 2, 3, 4, 5, 6]
n_neurons_list = [64, 128, 256, 512]

best_config = {
    'n_hidden_layers': -1,
    'n_neurons': -1,
    'loss': 100000,
    'model_history': None
}

Path("mlp-reg-models").mkdir(parents=True, exist_ok=True)

for n_hidden_layers in n_hidden_layers_list:
    for n_neurons in n_neurons_list:
        print('Training {} hidden layers with {} neurons'.format(n_hidden_layers, n_neurons))

        es = EarlyStopping(monitor='val_loss', patience=50)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001)
        mc = ModelCheckpoint('./mlp-reg-models/model-cp.h5', verbose=0, monitor='val_loss', save_best_only=True, mode="min")

        model = build_model((train_X.shape[1],), n_hidden_layers, n_neurons)
        history = model.fit(scaler.transform(train_X), train_y, validation_data=(scaler.transform(val_X), val_y), epochs=1000, verbose=0, callbacks=[es, reduce_lr, mc])

        model = load_model('./mlp-reg-models/model-cp.h5')
        loss = model.evaluate(scaler.transform(val_X), val_y, verbose=0)
        print('Training done. Val loss: {:.2f}'.format(loss[0]))

        if best_config['loss'] >= loss[0]:
            print('Best setup so far!')
            best_config['n_hidden_layers'] = n_hidden_layers
            best_config['n_neurons'] = n_neurons
            best_config['loss'] = loss[0]
            best_config['model_history'] = history
            model.save('./mlp-reg-models/model.h5')

In [ ]:
history = best_config['model_history']
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
test_X, test_y = utils.split_df_to_X_y(test_df)

In [ ]:
model = load_model('./mlp-reg-models/model.h5')
model.evaluate(scaler.transform(test_X), test_y)